In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [2]:
train_d = pd.read_csv(r'C:\Users\YATISH\Desktop\pucho\train.csv',header =None ,low_memory=False)

dff = train_d.copy()
dff.drop(0,inplace =True)
dff.reset_index(drop =True,inplace = True)

# taking 10000 entries only for all puposes
Y = dff.iloc[0:10000,0:1]
X = dff.iloc[0:10000,1:785]
Y_test = dff.iloc[10000:42000,0:1]
X_test = dff.iloc[10000:42000,1:785]

from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(categorical_features=[0])
Y = ohe.fit_transform(Y).toarray()
Y_test = ohe.fit_transform(Y_test).toarray()

C:\Users\YATISH\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
C:\Users\YATISH\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:390: DeprecationWarning: The 'categorical_features' keyword is deprecated in version 0.20 and will be removed in 0.22. You can use the ColumnTransformer instead.
  "use the ColumnTransformer instead.", DeprecationWarning)
C:\Users\YATISH\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:368: FutureWarning: The handling of

In [ ]:
#train_d = pd.read_csv(r'C:\Users\YATISH\Desktop\pucho\train.csv',header =None ,low_memory=False)
#test_d = pd.read_csv(r'C:\Users\YATISH\Desktop\pucho\test.csv',header = None , low_memory =False)
#sub_file = pd.read_csv(r'C:\Users\YATISH\Desktop\pucho\sample_submission.csv')

#dff = train_d.copy()
#dff.drop(0,inplace =True)
#dff.reset_index(drop =True,inplace = True)

#dft = test_d.copy()
#dft.drop(0,inplace =True)
#dft.reset_index(drop =True,inplace = True)

#Y = dff.iloc[0:42000,0:1]
#X = dff.iloc[0:42000,1:785]
#X_test = dft.iloc[0:28000,0:784]

#from sklearn.preprocessing import OneHotEncoder
#ohe = OneHotEncoder(categorical_features=[0])
#Y = ohe.fit_transform(Y).toarray()

In [ ]:
#visualizing dataset
first = X[3:4]
im = np.array(first,dtype = 'float')
im = im.reshape(28,28)
plt.imshow(im)
plt.show()

In [3]:
# setting up dimensioins of layers
input_width = 28
input_height = 28
input_channels = 1
input_pixels = 784

n_conv1 = 32
n_conv2 = 64
stride_conv1 = 1
stride_conv2 = 1
conv1_k = 5
conv2_k = 5
max_pool1_k = 2
max_pool2_k = 2

n_hidden = 1024
n_output = 10

input_size_to_hidden_layer = (input_width//(max_pool1_k*max_pool2_k))*(input_height//(max_pool1_k*max_pool2_k))*n_conv2

In [4]:
# assigning weights and biases
weights = {
    "wc1" : tf.Variable(tf.random_normal([conv1_k,conv1_k,input_channels,n_conv1])),
    "wc2" : tf.Variable(tf.random_normal([conv2_k,conv2_k,n_conv1,n_conv2])),
    "whl" : tf.Variable(tf.random_normal([input_size_to_hidden_layer,n_hidden])),
    "wo"  : tf.Variable(tf.random_normal([n_hidden,n_output]))    
}
biases = {
    "bc1" : tf.Variable(tf.random_normal([n_conv1])),
    "bc2" : tf.Variable(tf.random_normal([n_conv2])),
    "bhl" : tf.Variable(tf.random_normal([n_hidden])),
    "bo"  : tf.Variable(tf.random_normal([n_output]))    
}

Instructions for updating:
Colocations handled automatically by placer.


In [5]:
def conv(x,weights,biases,strides):
    out = tf.nn.conv2d(x, weights, padding = "SAME", strides = [1, strides, strides, 1])
    out = tf.nn.bias_add(out,biases)
    out = tf.nn.relu(out)
    return out

def maxpool(m, k =2):
    return tf.nn.max_pool(m, padding ="SAME", ksize = [1, k, k, 1], strides = [1, k, k, 1])

In [6]:
def cnn(x,weights,biases):
    x = tf.reshape(x, shape = [-1, input_width, input_height, input_channels])
    conv1 = conv(x,weights['wc1'],biases['bc1'],stride_conv1)
    conv1_pool = maxpool(conv1,max_pool1_k)
    
    conv2 = conv(conv1_pool,weights['wc2'],biases['bc2'],stride_conv2)
    conv2_pool = maxpool(conv2,max_pool2_k)
    
    hidden_layer_input = tf.reshape(conv2_pool,shape = [-1,input_size_to_hidden_layer])
    hidden_layer_output_before_activation = tf.add(tf.matmul(hidden_layer_input,weights['whl']),biases['bhl'])
    hidden_layer_output = tf.nn.relu(hidden_layer_output_before_activation)
    
    out_before_add = tf.matmul(hidden_layer_output,weights['wo'])
    output = tf.add(out_before_add,biases['bo'])
    
    return output

In [7]:
x = tf.placeholder("float",[None,input_pixels])
y = tf.placeholder("float",[None,n_output])
pred = cnn(x,weights,biases)

In [8]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits = pred,labels = y))

In [9]:
optimizer = tf.train.AdamOptimizer(learning_rate = 0.01)
optimize = optimizer.minimize(cost)

In [10]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [11]:
# batch gradient
batch_size = 100
for i in range(25):
    n_batches = len(X)//batch_size
    total_cost = 0
    b = 1
    for j in range(n_batches):
        b = batch_size*j
        batch_X = X[b:b+batch_size]
        batch_Y = Y[b:b+batch_size]
        c, _ = sess.run([cost,optimize],feed_dict ={x:batch_X,y:batch_Y})
        total_cost += c
    print(total_cost)

345551804.78125
5880174.5322265625
2452766.4937286377
1010034.9453735352
684779.1800079346
433427.8554325104
330668.7212982178
390390.6194381714
415317.56494140625
556683.8142852783
383858.9620437622
287009.3519973755
218088.94874572754
262017.4090423584
293978.9033203125
245299.69150924683
470864.84506225586
407129.88024902344
370571.5139160156
698112.9031677246
881521.277267456
527078.6349487305
553297.9660263062
458498.949798584
600582.9779968262


In [12]:
predictions = tf.argmax(pred,1)
true_labels = tf.argmax(Y_test,1)
correct_predictions = tf.equal(predictions,true_labels)

pred_eval,labels,correct_pred = sess.run([predictions,true_labels,correct_predictions] , feed_dict = {x:X_test,y:Y_test})
c_num = correct_pred.sum()

sess.close()
print("accuracy = ",(c_num*100)/len(X_test))

accuracy =  96.05625


In [ ]:
'''predictions = tf.argmax(pred,1)

pred_eval = sess.run(predictions , feed_dict = {x:X_test})

for i in range(len(sub_file)):
    sub_file.Label[i] = pred_eval[i]
sub_file.to_csv(r'C:\Users\YATISH\Desktop\pucho\submission_2.csv',index = False)'''